# IMPORT

In [2]:
# import 

import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

In [13]:
joblib.load("../../case/NDCG/Jac/hybridTerbaik/5/5_10_1_ndcg.joblib")

,NDCG@1,NDCG@2,NDCG@3,NDCG@4,NDCG@5,NDCG@6,NDCG@7,NDCG@8,NDCG@9,NDCG@10,NDCG@11,NDCG@12,NDCG@13,NDCG@14,NDCG@15,NDCG@16,NDCG@17,NDCG@18,NDCG@19,NDCG@20
User-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
User-924,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037791,0.036439,0.035201,0.034063
User-925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-926,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.049044,0.046807,0.044810,0.043015,0.041389,0.039909,0.038554,0.037307


# DATASET

In [3]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    test_File = f"{baseFile}/u{fold_Number}.test"
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    test_Data = test_Data.drop(columns=["timestamp"])
    return test_Data

In [5]:
call_base = "../../ml-100k"
testdata = load_Dataset(call_base, 1)
testdata


,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [9]:
def uniqDataTest(ratingData):
    unique_users = sorted(ratingData['user_id'].unique())
    unique_items = sorted(ratingData['item_id'].unique())

    # Grup item_id per user_id jadi list item (ground truth)
    grouped = ratingData.groupby('user_id')['item_id'].apply(list)

    # Buat list groundTruth (list item per user) sesuai urutan unique_users
    groundTruth = [grouped[user] for user in unique_users]

    return groundTruth


# RUMUS PRECISION

In [20]:
def Precision(GroundTruth, TopN, N):
    rumusPrecision = len(np.intersect1d(TopN[:N], GroundTruth)) / N
    # print(len(np.intersect1d(TopN[:N], GroundTruth)))
    # print((np.intersect1d(TopN[:N], GroundTruth)))
    # print(rumusPrecision)
    return rumusPrecision

In [21]:
def PrecisionAll(groundTruth, TopNrek, max_N):
    pres_matrix = []  # Menyimpan DCG tiap user untuk N = 1 sampai max_N

    for i in range(len(groundTruth)):
        user_pres = []
        # print(f"\nUser ke-{i+1}:")
        for N in range(1, max_N + 1):
            pres_value = Precision(groundTruth[i], TopNrek[i], N)
            user_pres.append(pres_value)
            # print(f"  Presision@{N}: {pres_value}")
        pres_matrix.append(user_pres)

    # Buat DataFrame
    columns = [f"Presision@{N}" for N in range(1, max_N + 1)]
    df_pres = pd.DataFrame(pres_matrix, columns=columns)
    df_pres.index = [f"User-{i+1}" for i in range(len(groundTruth))]

    # Tambahkan rata-rata press@N
    avg_row = df_pres.mean()
    df_pres.loc["Rata-rata"] = avg_row

    # print("\nDataFrame pres per user:")
    # print(df_dcg)

    return df_pres


# LOOP SEMUA FOLD

In [22]:
# LOOP SEMUA FOLD PRECSION
def LoopPrecisionAll(base_input_dir, base_output_dir, dataset_dir, N=20):
    for fold in range(1, 6):
        print(f"\n🔄 Processing Fold {fold}...")

        # Load calTest dan convert ke list ground truth
        testdata = load_Dataset(dataset_dir, fold)
        dataTestUniq = uniqDataTest(testdata)

        fold_input_path = os.path.join(base_input_dir, str(fold))
        fold_output_path = os.path.join(base_output_dir, str(fold))
        os.makedirs(fold_output_path, exist_ok=True)

        # Loop semua file .joblib di folder fold ini
        for filename in os.listdir(fold_input_path):
            if filename.endswith(".joblib"):
                filepath = os.path.join(fold_input_path, filename)
                try:
                    print(f"  Processing: {filename}")
                    openTopN = joblib.load(filepath)
                    # call ndcg fungsi 
                    callPrecision = PrecisionAll(dataTestUniq, openTopN, N)

                    # Simpan hasil DCGN
                    result_name = filename.replace(".joblib", "_precision.joblib")
                    result_path = os.path.join(fold_output_path, result_name)
                    joblib.dump(callPrecision, result_path)
                    print(f"     Saved to: {result_path}")
                    # save excel 
                    excel_path = result_path.replace(".joblib", ".xlsx")
                    callPrecision.to_excel(excel_path)
                    print(f"     Saved to Excel: {excel_path}")

                except Exception as e:
                    print(f"  Error processing {filename}: {e}")

## JAC

In [25]:
# calProses1 = LoopPrecisionAll("../../case/topN/Jac/topNUser", "hasil/Jac/user","../../ml-100k",N=20)

In [24]:
joblib.load("hasil/Jac/user/1/5_20_0.7_precision.joblib")

,Presision@1,Presision@2,Presision@3,Presision@4,Presision@5,Presision@6,Presision@7,Presision@8,Presision@9,Presision@10,Presision@11,Presision@12,Presision@13,Presision@14,Presision@15,Presision@16,Presision@17,Presision@18,Presision@19,Presision@20
User-1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-2,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-4,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-5,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
User-456,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-457,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-458,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
User-459,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
# calProses2 = LoopPrecisionAll("../../case/topN/Jac/topNItem", "hasil/Jac/item","../../ml-100k",N=20)

## RJ

In [30]:
# calProses3 = LoopPrecisionAll("../../case/topN/RJ/topNUser", "hasil/RJ/user","../../ml-100k",N=20)

In [31]:
# calProses4 = LoopPrecisionAll("../../case/topN/RJ/topNItem", "hasil/RJ/item","../../ml-100k",N=20)

## HYBRID